# Комментарий:
Предоставляю решение первого задания. При решении второго, столкнулся с проблемой реализации, так как я не смог найти open-source решения на python для разделения на различные голоса. Сложность заключается в том, что тут необходимо взаимодействовать с аудио-волнами, частотами, и если оба говорящих имеют +/- одинаковые голосовые показатели, то задача переходит в более сложную стадию и за 4 дня реализовать такое очень сложно. При анализе источников и решений, я нашел решение у СБЕРа, по разделению на голоса, но доступа к данной технологии у меня нету, так как необходима подписка.
На данный момент у меня имеется решение для распознования речи и перевод ее в текст (V2T), решения по второй части имеется на половину (Без разделения на персоны), но я готов заняться данной задачей в вашей компании (Если необходимо будет внедрить решение с распознованием персон)

#Установка зависимостей

> Добавить блок с цитатой



In [ ]:
!pip install audio-separator
!pip cache purge
!pip install --force-reinstall torch torchvision torchaudio
!pip install --force-reinstall onnxruntime-gpu
!pip install -U git+https://github.com/jianfch/stable-ts.git

# Модель whisper-3

Выбор данной модели основывался на ее хорошем распозновании текста из голосового файла, также хорошим бонусом является разделение не только на текст, но и на таймкоды.

Можно рассмотреть и другие модели, но данная модель является хорошей в своем сегменте. Еще не мало важным фактором является то, что она имеет исходный открытый код и не требует никаких токенов , по сравнению с аналагами.

Wisper - данную модель имеет несколько размеров параметров, и в зависимости от выбора размерности и будет зависеть качество. Отличным бонусом в данной модели является то, что мы получаем таймкоды, что дает хорошие возможности при обработки аудио файлов.

В пример кода не привожу никакие иные модели, так как они требуют либо токен, либо мощностей для развертывания, которых у меня не хватает. Список моделей, которые были использованы в ходе тестов - pyannote/speaker-diarization, facebook/seamless-m4t-v2-large




In [ ]:
from audio_separator.separator import Separator
from pydub import AudioSegment
import os


#@title Загружаем модель whisper-3

import stable_whisper
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Устройтво", device)

# Загрузка модели для транскрибации аудиофайлов,
# используя архитектуру "large-3",
# скачивание модели, если она не была предварительно загружена,
# и указание корневой директории для загрузки

if str(device) == "cuda":
    whisper_model = stable_whisper.load_model(
        'large-v3',
        device = "cuda",
        )
else:
    whisper_model = stable_whisper.load_model(
        'large-v3',
        device = device,
        in_memory=True,
        dq=True
        )
#@title Функция для транскребации голоса
import os


def transcribe(input_file):
    # Получаем имя файла без расширения из полного пути к input_file
    input_filename = os.path.splitext(os.path.basename(input_file))[0]

    # Создаем папку для сохранения аудиофайлов, если она не существует
    output_folder = f"{input_filename}"
    os.makedirs(output_folder, exist_ok=True)

    # Выполняем транскрибацию аудиофайла
    # с использованием модели stable_whisper
    result = whisper_model.transcribe(
        os.path.join(output_folder, f"{input_filename}.mp3"),
        )

    # Сохраняем результаты в формате TSV в файл
    # с именем, аналогичным названию входного файла
    result.save_as_json(os.path.join(output_folder, f"{input_filename}.json"))

    return "Done"

Устройтво cpu


100%|█████████████████████████████████████| 2.88G/2.88G [01:46<00:00, 29.0MiB/s]


# Запускаем процесс транскрибации

In [ ]:
transcribe("audio.mp3")

Transcribe:   0%|          | 0/132.34 [00:43<?, ?sec/s]

Detected language: russian


Transcribe: 100%|██████████| 132.34/132.34 [06:10<00:00,  2.80s/sec]

Saved: /content/audio/audio.json


'Done'